In [ ]:
%pip install elasticsearch==7.17
%pip install elasticsearch_dsl

In [423]:
from elasticsearch import Elasticsearch
import re
import os
import pandas as pd
import string
from elasticsearch_dsl import Search
import json

In [20]:
es = Elasticsearch("http://localhost:9200", timeout=60)

In [37]:
os.chdir("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA")
files = os.listdir("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/ap89_collection")
files.remove("readme")

In [40]:
# function (also used in parser/indexer) to get list of docnums
def get_docnums():
    test_keys = []
    for i in files[:364]:
        current = open(str("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/ap89_collection/"+i), encoding = "latin-1")
        text = current.read()
        keys = re.findall("<DOCNO>(.*)</DOCNO>", text)
        for j in keys:
            test_keys.append(j)
    return test_keys

In [22]:
# helper function for get_queries() to return the number of each query
def get_queries_nums(queries_list):
    queries_nums = []
    for line in queries_list:
        num = line.split()[0]
        queries_nums.append(num)
    return queries_nums

In [23]:
# helper function for get_queries() to return necessary text of each query with
def get_queries_text(queries_list):
    queries_nums = get_queries_nums(queries_list)
    query_stop_words = ["Document", "will", "must", "discuss", 
    "report", "include", "describe", "identify", "a", "an", "as",
    "and", "the", "to", "or", "either", "of", "by", "in", "with", 
    "about", "some", "any", "its", "even", "other", "which",
    "being", "certain", "has"]
    remove_words = queries_nums + query_stop_words
    queries_text = []
    for i in queries_list:
        x = i.split()
        queries_text.append(" ".join(a if a not in remove_words else '' for a in x))
    return queries_text

In [24]:
# function to get dataframe with query number and query text
def get_queries_df():
    queries_df = pd.DataFrame()
    with open("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/query_desc.51-100.short.txt") as queries_file:
        queries_list = queries_file.read()
        queries_list = re.sub("-", " ", queries_list)
        queries_list = queries_list.translate(str.maketrans('', '', string.punctuation))
        queries_list = queries_list.split("\n")
        queries_nums = get_queries_nums(queries_list)
        queries_text = get_queries_text(queries_list)
        queries_df["QueryNumber"] = queries_nums
        queries_df["QueryText"] = queries_text
    return queries_df

#queries_df = get_queries_df()
#print(queries_df)

In [25]:
# function to get list of queries 
def get_queries_words(queries_df):
    query_words_list = []
    for query in queries_df.iloc[:, 1]:
        query_words_list.append(query)
    return query_words_list

#queries_df = get_queries_df()
#query_words_list = get_queries_words(queries_df)
#print(query_words_list)


In [26]:
# helper function to get the body from of the queries for elasticsearch search
def get_query_bodies(query_words_list):
    queries_df = get_queries_df()
    q_nums_list = queries_df.iloc[:, 0]

    query_body_df = pd.DataFrame()
    q_body_list = []
    for query in query_words_list:
        query_body = {
          "query": {
              "match": {
                  "TEXT": query
              }
          }
        }
        q_body_list.append(query_body)
    query_body_df["QueryNumber"] = q_nums_list
    query_body_df["QueryBody"] = q_body_list

    return query_body_df


In [27]:
# main function runs all 25 queries and returns list of results from each search.
def run_queries():
    queries_df = get_queries_df()
    query_words_list = get_queries_words(queries_df)
    query_body_df = get_query_bodies(query_words_list)
    
    query_result_list = []
    for index, row in query_body_df.iterrows():
        q_body = query_body_df["QueryBody"][index]
        result = es.search(index="ap89_index4", body=q_body, size=1000)
        q_num = query_body_df["QueryNumber"][index]
        query_result_list.append([q_num, result])
  
    return query_result_list

In [28]:
# helper function gets list of docnums from query result (json)
def get_result_docnums(result):
    docnums = []
    for hit in result["hits"]["hits"]:
        docnum = hit["_id"]
        docnums.append(docnum)
    return docnums

In [430]:
# helper function gets list of scores from query result (json)
def get_result_scores(result):
    scores = []
    for hit in result["hits"]["hits"]:
        score = hit["_score"]
        scores.append(score)
    return scores

In [31]:
# helper function to get number of documents in a query result
def get_result_count(result):
    count = 0
    for hit in result["hits"]["hits"]:
        count = count + 1

    return count

In [385]:
# main function for ES Built-in queries - runs queries and generates output file
def es_built_in():
    query_result_list = run_queries()

    query_result_docnums = []
    query_result_scores = []
    query_number_result = []
    for list in query_result_list:
        result = list[-1]
        query_result_docnums += get_result_docnums(result)
        query_result_scores += get_result_scores(result)

        count = get_result_count(result)
        i = 0
        while i in range(count):
            query_number_result.append(list[0])
            i += 1

    with open('/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/ES_built_in_results.txt', 'w') as queryResults:
        rank = 1
        j = 0
        while j in range(len(query_result_docnums)):
            queryResults.write('%s Q0 %s %s %s Exp\n' % (query_number_result[j], query_result_docnums[j], rank, query_result_scores[j]))
            rank += 1
            if rank == 1001:
                rank = 1
            j += 1
    queryResults.close()

es_built_in()

/var/folders/4l/8brx29c94lq1trgx5m8_tz800000gp/T/ipykernel_58364/132238449.py:10: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="ap89_index4", body=q_body, size=1000)
